## This part calculates the posterior predictions for the present buckwheat distribution, counterfactual plots, and past distribution (add)

In [ ]:
library(rgdal)
### Set working directory:
setwd("/export/home/mk843/mnt/")
#setwd("G:\\My Drive\\SDM_China\\")

### Define the size of the sample from the posterior:
n=40000
n_cnt=1000

### Define the model to run
#mname<-'parabolic_interactions_iCAR'
mname<-'parabolic_simple_iCAR'
stan_model<-paste('stan_models/',mname,'.stan',sep="")

### Source the functions:

source("R/square.r")
source("R/calculatePredictions.r")

### Define the paths
path2model<-paste('outputs/models/',mname,'.rds',sep="")
path2predictions <- paste('outputs/05_01_Posterior_Predictions_',mname,'.csv',sep="")
path2counterfactual <-paste('outputs/05_02_Counterfactual_predictions_',mname,'.csv',sep="")
path2future <- paste('outputs/05_03_Future_predictions_',mname,"_",sep="")
path2pred_summary <- paste('outputs/05_04_Predictions_summary_',mname,".csv",sep="")


### Load reqired data:
var_ordered<-c('BIO10','BIO17','BIO4','BIO9','npp')
china<-readOGR(dsn = "data/china_data", layer = "china_data")
data<-china@data[ paste(var_ordered,"sd",sep="_")]
fit<-readRDS(path2model)

### Extract posterior distribution for all parameters:
posterior <- rstan::extract(fit) 

### Calculate predictions
predictions<-calculatePredictions(fit, data, posterior,n,counterfactual=FALSE)

#### Calculate predictions summary:
predictions<-read.csv(path2predictions)
means_pred<-apply(predictions,2, mean)
quantiles_pred<-apply(predictions,2, quantile,probs = c(0.05, 0.95))
pred<-t(rbind(quantiles_pred, means_pred))
exp_pred<-exp(pred)
colnames(pred)<-paste(colnames(pred),"_log",sep="")
pred<-cbind(pred,exp_pred)


### Save predictions:

write.csv(predictions, path2predictions,col.names=FALSE, row.names=FALSE)

### Calculate the predictions for the future:

for (i in 1:6){
    print(i)
    data<-china@data[grep(paste("_",i,"_",sep=""),colnames(china@data))]
    data<-data[unlist(lapply(var_ordered,grep,colnames(data),value=TRUE))]
    predictions<-calculatePredictions(fit, data, posterior,n,counterfactual=FALSE)
    write.csv(predictions, paste(path2future,i,"000.csv",sep=""),col.names=FALSE, row.names=FALSE)
    ### Also calculate the summary
    means_pred<-apply(predictions,2, mean)
    quantiles_pred<-apply(predictions,2, quantile,probs = c(0.05, 0.95),na.rm=TRUE)
    fpred<-t(rbind(quantiles_pred, means_pred))
    exp_pred<-exp(fpred)
    colnames(exp_pred)<-paste(colnames(fpred),"_",i,"k",sep="")
    colnames(fpred)<-paste(colnames(fpred),"_",i,"k_log",sep="")
    fpred<-cbind(fpred,exp_pred)
    pred<-cbind(pred,fpred)
}

### Write predictions to the file:
write.csv(pred,path2pred_summary)
china@data<-cbind(china@data,pred)
writeOGR(obj=china, dsn="data/china_predictions", layer="china_predictions", driver="ESRI Shapefile")

In [ ]:
### Calculate the predictions for the posterior plots
### Also do counterfactual plots

means<-colMeans(data)
means<-as.data.frame(rbind(means))
means<-means[rep(seq_len(nrow(means)), each = n_cnt), ]
range<-apply(data,2,range)

for (i in 1:ncol(data)){
    print(i)
    new_data<-means
    new_data[,i]<-seq(from = range[1,i], to = range[2,i],length.out=n_cnt)
    predictions<-calculatePredictions(fit, new_data, posterior,n,counterfactual=TRUE)
    write.csv(predictions, gsub("predictions", paste("predictions",colnames(data)[i],sep="_"),path2counterfactual),col.names=FALSE, row.names=FALSE)
}



In [ ]:
### Calculate the summury of the predictions and future predictions:

##This loads future predictions
fp<-c()

for (i in 2:length(ffiles)){
    print(i)
    x<-read.csv(ffiles[i])
    fp[[i]]<-x    
    }
### Get summary for the future predictions

for(i in 1:length(fp)){
    fpred<-fp[[i]]
    means_pred<-apply(fpred,2, mean)
    quantiles_pred<-apply(fpred,2, quantile,probs = c(0.05, 0.95),na.rm=TRUE)
    fpred<-t(rbind(quantiles_pred, means_pred))
    exp_pred<-exp(fpred)
    colnames(exp_pred)<-paste(colnames(fpred),"_",i,"k",sep="")
    colnames(fpred)<-paste(colnames(fpred),"_",i,"k_log",sep="")
    fpred<-cbind(fpred,exp_pred)
    pred<-cbind(pred,fpred)
}



china@data<-cbind(china@data,pred)